<a href="https://colab.research.google.com/github/fberanizo/spelling-correction/blob/master/BEA2019_T5_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BEA 2019 Shared Task: Grammatical Error Correction: Modelo T5

**Nome: Fabio Beranizo Lopes**<br>
**Nome: Luiz Pita Almeida**

Usaremos o modelo T5 pré-treinado e o dataset W&I+LOCNESS do [Building Educational Applications 2019 Shared Task: Grammatical Error Correction](https://www.cl.cam.ac.uk/research/nl/bea2019st). <br>
Truncamos para strings de tamanho 100 para deixar os testes mais rápidos.

Métrica de avaliação: F0.5-score <br>
https://www.cl.cam.ac.uk/research/nl/bea2019st/#eval

O método de correção aplicado foi realizar fine-tuning em no modelo T5 para a tarefa de correção.

Passos:

1. Geram-se tuplas: `(original, corrected)`
2. Aplica-se modelo T5 para prever top-10 palavras.<br>
   Caso a palavra original esteja no Top 10 do modelo, é classificada como correta.<br>
   Senão, a palavra é classificada como incorreta.

**Obs: os notebooks contém excertos de códigos dos colegas de turma.**<br>
**Obrigado Diedre, Gabriela, Leard, Lucas e Israel.**


In [ ]:
import torch

print(f"Current GPU: {torch.cuda.get_device_name(0)}")

# don't even start if it's not a P100 GPU
# if torch.cuda.get_device_name(0) != "Tesla P100-PCIE-16GB":
#     import os
#     os.kill(os.getpid(), 9)

Current GPU: Tesla P100-PCIE-16GB


In [ ]:
#@title Configurações gerais
experiment_name = "fine-tuning" #@param {type:"string"}
model_name = "t5-small"  #@param ["t5-small", "t5-base", "t5-large", "t5-3B", "t5-11B"] {type:"string"}
max_epochs = 10 #@param {type:"integer"}
batch_size =  2 #@param {type:"integer"}
accumulate_grad_batches =   25#@param {type:"integer"}
sequence_length = 1000  #@param {type:"integer"}
learning_rate = 5e-3  #@param {type:"number"}
decode_mode = "topk"  #@param ["greedy", "nucleus", "topk", "beam"] {type:"string"}
k = 10  #@param {type:"integer"}

## Instala dependências

- PyTorch Lightning
- Hugginface Transformers
- ERRANT (ERRor ANnotation Toolkit)
- pyxDamerauLevenshtein

In [ ]:
try:
    import pytorch_lightning
    import transformers
except ImportError as e:
    # can't import modules, then install
    !pip install --quiet pytorch-lightning
    !pip install --quiet transformers
    !pip install --quiet errant==2.0.0
    !pip install pyxDamerauLevenshtein
    !python -m spacy download en
    # kill kernel (necessary for tqdm)
    import os
    os.kill(os.getpid(), 9)

In [ ]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.
import datetime
import errant
import gzip
import json
import numpy as np
import nvidia_smi
import os
import pandas as pd
import joblib
import psutil
import pytorch_lightning as pl
import random
import spacy
import sys
import tarfile
import tempfile
import torch
import torch.nn.functional as F

from argparse import Namespace
from google.colab import drive
from itertools import cycle

from pyxdameraulevenshtein import damerau_levenshtein_distance, \
    normalized_damerau_levenshtein_distance
from pyxdameraulevenshtein import damerau_levenshtein_distance_ndarray, \
    normalized_damerau_levenshtein_distance_ndarray

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from typing import Dict
from typing import List
from typing import Tuple

# Leard decoding solution
import html
import unicodedata

nlp = spacy.load("en")
annotator = errant.load("en", nlp)

nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

def hardware_stats():
    """
    Returns a dict containing some hardware related stats
    """
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    return {"cpu": f"{str(psutil.cpu_percent())}%",
            "mem": f"{str(psutil.virtual_memory().percent)}%",
            "gpu": f"{str(res.gpu)}%",
            "gpu_mem": f"{str(res.memory)}%"}

## Define random seeds

Importante: Fix seeds so we can replicate results

In [ ]:
import random

seed = 0
random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Mapeia Google Drive

Iremos salvar os checkpoints (pesos do modelo) no google drive, para que possamos continuar o treino de onde paramos.

In [ ]:
drive.mount("/content/drive")
base_path = "/content/drive/My Drive/PF-Correcao/bea2019st-finetuning"
# base_path = "/content/bea2019st-finetuning"
os.environ["BASE_PATH"] = base_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ERRANT Scorer

Comando para avaliação que compara um arquivo M2 "hipótese" contra um arquivo M2 "referência".<br>

### **Example**
**Original**: This are gramamtical sentence .<br>
**Corrected**: This is a grammatical sentence .<br>
**Output M2**:<br>
S This are gramamtical sentence .<br>
A 1 2|||R:VERB:SVA|||is|||REQUIRED|||-NONE-|||0<br>
A 2 2|||M:DET|||a|||REQUIRED|||-NONE-|||0<br>
A 2 3|||R:SPELL|||grammatical|||REQUIRED|||-NONE-|||0<br>
A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||1<br>

In M2 format, a line preceded by S denotes an original sentence while a line preceded by A indicates an edit annotation. Each edit line consists of the start and end token offset of the edit, the error type, and the tokenized correction string. The next two fields are included for historical reasons (see the CoNLL-2014 shared task) while the last field is the annotator id.

In [ ]:
%%writefile orig.txt
Eu não cei pra ondi vou .
Podi até num dá em nada .
Minha vida segui o sol .
No horizonti dessa istrada .

Overwriting orig.txt


In [ ]:
%%writefile ref.txt
Eu não sei pra onde vou .
Pode até não dar em nada .
Minha vida segue o sol .
No horizonte dessa estrada .

Overwriting ref.txt


In [ ]:
%%writefile hyp.txt
Eu não sei pra ondi vou .
Podi até não dar em nada .
Minha vida segui u sol .
Num horizonte dessa estrada .

Overwriting hyp.txt


In [ ]:
!errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
!errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null

In [ ]:
import pandas as pd
!errant_compare -hyp hyp.m2 -ref ref.m2
# x = !errant_compare -hyp hyp.m2 -ref ref.m2
# df = pd.DataFrame(data=x[2:4])[0].str.split('\t', expand=True)
# new_header = df.iloc[0] #grab the first row for the header
# df = df[1:] #take the data less the header row
# df.columns = new_header
# df
# df["F0.5"][1]
# d = df.apply(pd.to_numeric).to_dict('r')



=========== Span-Based Correction ============
TP	FP	FN	Prec	Rec	F0.5
5	2	3	0.7143	0.625	0.6944



## Gerador 

From:
https://github.com/huggingface/transformers/issues/3985

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)
# # Input text
# original = 'This are gramamtical sentence .'
# correct = 'This is a grammatical sentence .'

# text = 'This <extra_id_0> sentence. </s>'

# encoded = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
# input_ids = encoded['input_ids']

# # Generating 20 sequences with maximum length set to 5
# outputs = model.generate(input_ids=input_ids, 
#                           num_beams=200, num_return_sequences=20,
#                           max_length=5)

# _0_index = text.index('<extra_id_0>')
# _result_prefix = text[:_0_index]
# _result_suffix = text[_0_index+12:]  # 12 is the length of <extra_id_0>

# def _filter(output, end_token='<extra_id_1>'):
#     # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
#     _txt = tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
#     if end_token in _txt:
#         _end_token_index = _txt.index(end_token)
#         return _result_prefix + _txt[:_end_token_index] + _result_suffix
#     else:
#         return _result_prefix + _txt + _result_suffix

# results = list(map(_filter, outputs))
# results
# del tokenizer
# del model

## Classe Dataset
Gerenciamento dos dados, e um pequeno teste.

In [ ]:
hparams = {"model_name": model_name, "seq_len": sequence_length, "batch_size": batch_size}
class BEA2019(Dataset):
    """
    Loads data from preprocessed file and manages them.
    """
    VALID_MODES = ["train", "validation", "test"]
    TOKENIZER = T5Tokenizer.from_pretrained(hparams["model_name"],
                                            cache_dir=base_path)
    def __init__(self, mode: str, seq_len: int):
        """
        mode: One of train, validation or test 
        seq_len: limit to returned encoded tokens
        """
        super().__init__()
        assert mode in BEA2019.VALID_MODES

        self.mode = mode
        self.seq_len = seq_len

        file_name = os.path.join(base_path, f"{mode}.pkl")
        if not os.path.isfile(file_name):
            print("Pre-processed files not found, preparing data.")
            self.prepare_data()
        
        with open(file_name, "rb") as preprocessed_file:
            self.data = joblib.load(preprocessed_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i: int):
        """
        Unpacks line from data and applies T5 encoding if necessary.

        returns: input (corrupted, original), target (corrected)
        """
        input, target = self.data[i]
        
        input_normalized = unicodedata.normalize("NFD", input).\
            encode("latin-1", "xmlcharrefreplace").\
            decode("latin-1")

        input_encoded = BEA2019.TOKENIZER.encode_plus(
            f"{input_normalized} {BEA2019.TOKENIZER.eos_token}",
            max_length=self.seq_len,
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_tensors="pt")

        input_ids = input_encoded["input_ids"].squeeze()
        input_mask = input_encoded["attention_mask"].squeeze()

        target_normalized = unicodedata.normalize("NFD", target).\
            encode("latin-1", "xmlcharrefreplace").\
            decode("latin-1")

        target_encoded = BEA2019.TOKENIZER.encode_plus(
            f"{target_normalized} {BEA2019.TOKENIZER.eos_token}",
            max_length=self.seq_len,
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_tensors="pt")

        target_ids = target_encoded["input_ids"].squeeze()
        target_mask = target_encoded["attention_mask"].squeeze()

        return input, target, input_ids, input_mask, target_ids, target_mask

    def get_dataloader(self, batch_size: int, shuffle: bool):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, 
                          num_workers=4)

    @staticmethod
    def load_text_tuples(path, member):
        """
        Load tuples from original files: text_original, text_corrected.
        """
        text_tuples = []
        with tarfile.open(path) as tar:
            f = tar.extractfile(member)
            for line in f:
                data = json.loads(line)
                # list of edits, one for each annotators
                for annotator_id, edits in data["edits"]:
                    # edit: [annotator_id, [[char_start_offset, char_end_offset, correction], ...]]

                    text_original = data["text"]
                    text_corrected = ""
                    prev_char_end_offset = 0

                    for idx, (char_start_offset, char_end_offset, correction) in enumerate(edits):
                        # a slice of unchanged original text 
                        text_unchanged = text_original[prev_char_end_offset:char_start_offset]

                        if correction is None:
                            correction = ""
                        text_corrected = f"{text_corrected} {text_unchanged}{correction}"

                        prev_char_end_offset = char_end_offset

                    text_unchanged = text_original[prev_char_end_offset:]
                    text_corrected = f"{text_corrected} {text_unchanged}".lstrip()
                    text_tuples.append((text_original, text_corrected))

        return text_tuples

    @staticmethod
    def prepare_data(train_size=1000, val_size=300, test_size=300):
        """
        Performs everything needed to get the data ready.
        Addition of Eos token and encoding is performed in runtime.
        """
        if not os.path.isfile("wi+locness_v2.1.bea19.tar.gz"):    
            !wget -nc https://www.cl.cam.ac.uk/research/nl/bea2019st/data/wi+locness_v2.1.bea19.tar.gz -P "$BASE_PATH"
            # !wget -nc https://www.cl.cam.ac.uk/research/nl/bea2019st/data/ABCN.bea19.dev.orig -P "$BASE_PATH"
            # !wget -nc https://www.cl.cam.ac.uk/research/nl/bea2019st/data/ABCN.bea19.test.orig -P "$BASE_PATH"

        data = {}
        train_val_data = BEA2019.load_text_tuples(os.path.join(base_path, "wi+locness_v2.1.bea19.tar.gz"), "wi+locness/json/A.train.json")
        test_data = BEA2019.load_text_tuples(os.path.join(base_path, "wi+locness_v2.1.bea19.tar.gz"), "wi+locness/json/A.dev.json")

        random.shuffle(train_val_data)

        train_data = train_val_data[:train_size]
        val_data = train_val_data[train_size:train_size + val_size]
        test_data = test_data[:test_size]

        for mode, data in zip(BEA2019.VALID_MODES, [train_data, val_data, test_data]):
            file_name = os.path.join(base_path, f"{mode}.pkl")
            with open(file_name, "wb") as pkl_file:
                joblib.dump(data, pkl_file)
            print(f"Pre-processed data saved as {file_name}.")


datasets = {m: BEA2019(mode=m, seq_len=hparams["seq_len"]) for m in BEA2019.VALID_MODES}

# Testing datasets
for mode, dataset in datasets.items():
    print(f"\n{mode} dataset length: {len(dataset)}\n")
    print("Random sample")
    input, target, input_ids, input_mask, target_ids, target_mask = random.choice(dataset)
    print("input\n", input, end="\n\n")
    print("target\n", target, end="\n\n")


train dataset length: 1000

Random sample
input
 At present, many people think that English is the world language and it is absolutely an essential skill to communicate with other countries people. Due to this reason , Taiwanese parents do not want their kids to start behind others, thus, many kindergartens started to teach children English.


target
 At present, many people think that English is the world language and it is absolutely an essential skill to communicate with people from other countries . For  this reason,  Taiwanese parents do not want their kids to be  behind others. Therefore , many kindergartens have  started to teach children English.



validation dataset length: 300

Random sample
input
 No future for public transport?
why do use public transport some of us use it to save money for not buying gas or they can't buy a car or if they had problems in their cars or they don't have the mood to drive.
Now a days with technology we have different ways to transport like h

## Dataloaders

Verificação se dataloaders estão funcionando corretamente.

In [ ]:
shuffle = {"train": True, "validation": False, "test": False}
debug_dataloaders = {mode: datasets[mode].get_dataloader(batch_size=hparams["batch_size"], 
                                                         shuffle=shuffle[mode])
                     for mode in BEA2019.VALID_MODES}

# Testing dataloaders
for mode, dataloader in debug_dataloaders.items():
    print("{} number of batches: {}".format(mode, len(dataloader)))
    batch = next(iter(dataloader))

train number of batches: 500
validation number of batches: 150
test number of batches: 65


## Lightning Module

Aqui a classe principal do PyTorch Lightning é definida.


In [ ]:
class T5Corrector(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        self.t5 = T5ForConditionalGeneration.from_pretrained(hparams.model_name,
                                                             cache_dir=hparams.base_path)
        self.tokenizer = BEA2019.TOKENIZER
        self.start_token = BEA2019.TOKENIZER.convert_tokens_to_ids('<extra_id_0>')
        self.end_token = BEA2019.TOKENIZER.convert_tokens_to_ids('<extra_id_1>')

    def forward(self, x):
        inputs, targets, input_ids, input_mask, target_ids, target_mask = x

        if self.training:
            outputs = self.t5(input_ids=input_ids, 
                              attention_mask=input_mask,
                              lm_labels=target_ids)
            loss, predicted_scores = outputs[:2]
            return loss, predicted_scores, inputs, targets
        else:
            outputs = self.t5.generate(input_ids=input_ids, 
                                       attention_mask=input_mask,
                                       lm_labels=target_ids,
                                       max_length=self.hparams.seq_len)
            return outputs, inputs, targets

    def training_step(self, batch, targets):
        loss, predicted_scores, inputs, targets = self(batch)

        return {"loss": loss, "log": {"loss": loss}, "progress_bar": hardware_stats()}

    def validation_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)

        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            )
            for output_ids in outputs]

        with open("orig.txt", "w") as f:
            for input in inputs:
                input = input.replace("\n", "")
                f.write(f"{input}\n")

        with open("ref.txt", "w") as f:
            for target in targets:
                target = target.replace("\n", "")
                f.write(f"{target}\n")

        with open("hyp.txt", "w") as f:
            for pred in predicts:
                pred = pred.replace("\n", "")
                f.write(f"{pred}\n")

        !errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
        !errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null
        x = !errant_compare -hyp hyp.m2 -ref ref.m2
        df = pd.DataFrame(data=x[2:4])[0].str.split("\t", expand=True)
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:].apply(pd.to_numeric) #take the data less the header row
        df.columns = new_header

        true_positive = df["TP"][1]
        false_positive = df["FP"][1]
        false_negative = df["FN"][1]
        precision = df["Prec"][1]
        recall = df["Rec"][1]
        f_score = df["F0.5"][1]

        progress_bar = hardware_stats()
        progress_bar.update({"precision": precision, "recall": recall, "f_score": f_score})

        return {"true_positive": true_positive, "false_positive": false_positive, "false_negative": false_negative,
                "precision": precision, "recall": recall, "f_score": f_score,
                "predicts": predicts, "inputs": inputs, "targets": targets, "progress_bar": progress_bar}

    def test_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)

        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            )
            for output_ids in outputs]

        with open("orig.txt", "w") as f:
            for input in inputs:
                input = input.replace("\n", "")
                f.write(f"{input}\n")

        with open("ref.txt", "w") as f:
            for target in targets:
                target = target.replace("\n", "")
                f.write(f"{target}\n")

        with open("hyp.txt", "w") as f:
            for pred in predicts:
                pred = pred.replace("\n", "")
                f.write(f"{pred}\n")

        !errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
        !errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null
        x = !errant_compare -hyp hyp.m2 -ref ref.m2
        df = pd.DataFrame(data=x[2:4])[0].str.split("\t", expand=True)
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:].apply(pd.to_numeric) #take the data less the header row
        df.columns = new_header

        true_positive = df["TP"][1]
        false_positive = df["FP"][1]
        false_negative = df["FN"][1]
        precision = df["Prec"][1]
        recall = df["Rec"][1]
        f_score = df["F0.5"][1]

        progress_bar = hardware_stats()
        progress_bar.update({"precision": precision, "recall": recall, "f_score": f_score})

        return {"true_positive": true_positive, "false_positive": false_positive, "false_negative": false_negative,
                "precision": precision, "recall": recall, "f_score": f_score,
                "predicts": predicts, "inputs": inputs, "targets": targets, "progress_bar": progress_bar}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        return {"log": {"train_loss": avg_loss}} 

    def validation_epoch_end(self, outputs):
        avg_precision = sum([x["precision"] for x in outputs]) / len(outputs)
        avg_recall = sum([x["recall"] for x in outputs]) / len(outputs)
        avg_f_score = sum([x["f_score"] for x in outputs]) / len(outputs)

        tensorboard_logs = {"avg_precision": avg_precision,
                            "avg_recall": avg_recall,
                            "avg_f_score": avg_f_score}

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")

        return {"avg_precision": avg_precision, "avg_recall": avg_recall, "avg_f_score": avg_f_score,
                "log": tensorboard_logs, "progress_bar": tensorboard_logs}

    def test_epoch_end(self, outputs):
        avg_precision = sum([x["precision"] for x in outputs]) / len(outputs)
        avg_recall = sum([x["recall"] for x in outputs]) / len(outputs)
        avg_f_score = sum([x["f_score"] for x in outputs]) / len(outputs)

        tensorboard_logs = {"avg_precision": avg_precision,
                            "avg_recall": avg_recall,
                            "avg_f_score": avg_f_score}

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")
        
        return {"avg_precision": avg_precision, "avg_recall": avg_recall, "avg_f_score": avg_f_score,
                "log": tensorboard_logs, "progress_bar": tensorboard_logs}

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.hparams.lr)

    def train_dataloader(self):
        if self.hparams.overfit_pct > 0:
            logging.info("Disabling train shuffle due to overfit_pct.")
            shuffle = False
        else:
            shuffle = True
        dataset = BEA2019("train", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=shuffle)

    def val_dataloader(self):
        dataset = BEA2019("validation", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=False)

    def test_dataloader(self):
        dataset = BEA2019("test", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=False)

## Preparação

In [ ]:
hparams = {"name": experiment_name, "base_path": base_path,
           "model_name": model_name, "seq_len": sequence_length,
           "decode_mode": decode_mode, "k": k,
           "lr": learning_rate, "batch_size": batch_size, "batch_accum": accumulate_grad_batches,
           "max_epochs": max_epochs,
           "overfit_pct": 0, "debug": 0,
           "decode_mode": decode_mode}


for key, parameter in hparams.items():
    print("{}: {}".format(key, parameter))

name: fine-tuning
base_path: /content/drive/My Drive/PF-Correcao/bea2019st-finetuning
model_name: t5-small
seq_len: 1000
decode_mode: topk
k: 10
lr: 0.005
batch_size: 2
batch_accum: 25
max_epochs: 10
overfit_pct: 0
debug: 0


In [ ]:
# Instantiate model
model = T5Corrector(Namespace(**hparams))

# Folder/path management, for logs and checkpoints
tensorboard_path = "logs"
experiment_name = hparams["name"]
model_folder = os.path.join(tensorboard_path, experiment_name)
os.makedirs(model_folder, exist_ok=True)
ckpt_path = os.path.join(model_folder, "-{epoch}")

# Callback initialization
checkpoint_callback = ModelCheckpoint(prefix=experiment_name, 
                                      filepath=ckpt_path, 
                                      mode="max")
logger = TensorBoardLogger(tensorboard_path, experiment_name)

# PL Trainer initialization
trainer = Trainer(gpus=1,
                  checkpoint_callback=checkpoint_callback, 
                  early_stop_callback=False,
                  logger=logger,
                  accumulate_grad_batches=hparams["batch_accum"],
                  max_epochs=hparams["max_epochs"], 
                  fast_dev_run=bool(hparams["debug"]), 
                  overfit_pct=hparams["overfit_pct"],
                  progress_bar_refresh_rate=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


## Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/My Drive/PF-Correcao/bea2019st-finetuning"

Reusing TensorBoard on port 6006 (pid 889), started 0:50:39 ago. (Use '!kill 889' to kill it.)

<IPython.core.display.Javascript object>

## Fine-tuning

In [ ]:
trainer.fit(model)


  | Name | Type                       | Params
----------------------------------------------------
0 | t5   | T5ForConditionalGeneration | 60 M  


\Input: Syria has a lot of good restaurants .
The Damascus restaurant at the centre of the city is quite a nice place to have a meal.
It has all kinds of food even the really fancy ones .
You can enjoy having your breakfast there with your family because they have too many types of food .
This restaurant also has games for children which they can spend their time playing it .
It also includes a huge parking so you can park your own car freely .
At Friday they serve special food for those who don't have enough money to pay .
It's a very nice restaurant which I prefer the most

Target: Syria has a lot of good restaurants. 
The Damascus restaurant at the centre of the city is quite a nice place to have a meal.
It has all kinds of food,  even  really fancy food  .
You can enjoy having your breakfast there with your family,  because they have so  many types of food. 
This restaurant also has games for children,  which they can spend their time playing with  .
It also includes a huge car par

\Input: I am writing this letter because we have to improve the gym of the school.

The gym has many problems that we are gouing to explain :

the first problem is that we don´t have enough aparatous for all of the students 

the second problem is about that same aparatous are not working well because the school didn´t the manteinace long time ago

For the first problem in my opinion , the school have to buy some others aparatus because there are not enough for all of the students

For the second problem the solution for me is to hire someone to do the maintenance and say to all of the students to take care of the machines of the school

I hope my proposal wiil be useful for you

I look forward to hearing from you soon

Sebastián Ferrari
Target: I am writing this letter because we need  to improve the school gym .

The gym has many problems that we are going  to describe : 

The  first problem is that we don´t have enough apparatus  for all of the students.  

The  second problem is ab

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: Can save best model only with val_loss available, skipping.
  warnings.warn(*args, **kwargs)


\Input: Dear Linda, 
Imy name is Lorenza. My height is 1.65 metres. My hair are browns and my eyes are dark. I will arrive in the Central Station at eleven o'clok and I will wear a read cot and a yellow scarf. See you later. Thank you very much. 
Lo
Target: Dear Linda, 
My  name is Lorenza. My height is 1.65 metres. My hair is  brown  and my eyes are dark. I will arrive at  the Central Station at eleven o'clock  and I will wear a read coat  and a yellow scarf. See you later. Thank you very much. 
Lo
Prediction: Imy name is Lorenza. My height is 1.65 metres. My hair are browns and my eyes are dark. I will arrive at the Central Station at eleven o'clok and I will wear a read cot and a yellow scarf. See you later. Thank you very much. Lorenza. Thank you very much. Lorenza. My height is 1.65 metres. My hair are browns and my eyes are dark. I will arrive at the Central Station at eleven o'clok and I will wear a read cot and a red scarf. See you later. Thank you very much.



\Input: How we treat the earth has a direct impact on ourselves. Today,because of our progress of industry, we produced so many environmental problems that hadn't appeared before. For examples, lack of purity water, shortage of electricity, climate change, natural source depletion,and etc. The problems weren't so significant that we overlooked them, but now they become  serious threats that we can't ignore. How can we solve these problems?  Just change your daily habits. 

Target: How we treat the earth has a direct impact on ourselves. Today, because  of the  progress of industry, we have caused  so many environmental problems that did n't arise  before. For example , lack of clean  water, shortage of electricity, climate change, natural resource  depletion,  etc. The problems weren't very  significant so  we overlooked them, but now they have  become  serious threats that we can't ignore. How can we solve these problems?  Just change your daily habits. 

Prediction: Nowadays, because

\Input: Hello Julia,
How are you?
What did you do yestreday?
I went to the cinema to watch a comedy with my friends yesterday, but I didn't like it because it was so boring.
I prefer another kind of films than comedy, but yesterday there weren't any good film at the cinema.
what kind of films do you like?
I like thriller and the action films because they have never been boring.
Do you prefer to watch films at home or go to the cinema?
I prefer to watch films at the cinema because there aren't any distractions or noises and the quality of films are better.

See you soon.
Erica 
Target: Hello Julia,
How are you?
What did you do yesterday ?
I went to the cinema to watch a comedy with my friends yesterday, but I didn't like it because it was so boring.
I prefer other  kinds  of films than comedy, but yesterday there weren't any good films  at the cinema.
What  kind of films do you like?
I like thrillers  and the action films because they are never  boring.
Do you prefer watching  films at 

In [ ]:
trainer.test(model)